In [3]:
import os, json, torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 안전하게 경로 설정
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    base_dir = os.getcwd()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 예측 ID를 해석하려면 (선택사항)
label_map = {
    0: "졸업요건",
    1: "학교 공지사항",
    2: "학사일정",
    3: "식단 안내",
    4: "통학/셔틀 버스",
}

# 모델과 토크나이저 불러오기
model_dir = os.path.join(base_dir, "roberta")
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model     = AutoModelForSequenceClassification.from_pretrained(model_dir)
model.to(device).eval()

# 입력 데이터 로드
test_path = os.path.join(base_dir, "..", "data", "cls_test.json")
with open(test_path, "r", encoding="utf-8") as f:
    samples = json.load(f)

outputs = []

# 예측 반복
for entry in samples:
    text = entry["question"]

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits

    pred_id = int(logits.argmax(dim=-1).cpu())

    # 결과 저장
    outputs.append({
        "question": text,
        "label": pred_id,  # 또는 label_map[pred_id] 써도 됨
    })

# 결과 저장
out_path = os.path.join(base_dir, "outputs", "cls_output.json")
os.makedirs(os.path.dirname(out_path), exist_ok=True)  # outputs 폴더 없으면 생성

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(outputs, f, ensure_ascii=False, indent=2)

print(f">>> 예측 결과 저장 완료: {out_path}")

>>> 예측 결과 저장 완료: D:\lastproject\Termproject_{9조}\src\outputs\cls_output.json
